# Engineering Notebook: Design Exercise 1

### Ideas for the Wire Protocol

- Use TCP/IP as template
    - Do we need IP addresses in the header?
    - How can we verify packets?
- VBS: Should we use some kind of compression for the message contents?
    - VBS: Maybe not, let's keep it simple
- Header (8 bytes):
    - Version (1 byte)
    - Request code (think: HTTP status code but also use this to specify the operation) (1 byte)
    - Flags/status (2 bytes) -> for the future, also padding
    - Packet checksum (2 bytes)
    - Payload length (2 bytes)

- What type of data are we sending across?
    - Operations + arguments (e.g. page/pagination, optional arguments)
    - Strings (e.g. name, password hash, error messages)
    - Status code (from the server to detect errors)
    - Arrays of strings -> could be sent as string of strings separated by commas like a CSV
        - VBS: we need to allow for commas in a string, should we just use ascii and then have a special non-ascii character for either both end of string and end of item or a separate one for end of item and end of string, kind of like '\0' in C?
        - Let's not use this. Let's use another, more flexible encoding instead
    - Assuming that we never need to send over dictionaries but at most lists, we can use an encoding for "fields" like this:
        - Field ID (1 byte), Field Length (2 bytes), Field Value (bytes)
        - This would mean that the client and server need to agree on which field ID corresponds to which field (e.g. username -> ID 1 etc.), and -- importantly -- what the *type* of each field is (e.g. float, string, unsigned int/int, list of float/string etc.); also, there are only 2^8 = 255 possible fields we can use which should be enough for this application but may cause issues in the future if there is a lot of data that we need to transmit
            - One possible workaround for this is to have an "OTHER" value kind of like in UTF-8 where if the first bit is set, you actually read 2 bytes for the field ID and use that as your index; this would be much more flexible but also more work to implement
        - Do we want to allow lists of different types? Probably not necessary
        - Side note: we can store the types efficiently using some byte flag for "list" and then different byte values for the type, and then check if "list & type" is 1 to detect that it's a list
        - We should agree that we're using little-endian for all communication
            - Is there any way to define flexible length types, e.g. specify "int" but then allow 8-byte integers?
    - This doesn't let us send lists of lists or lists of dictionaries or lists of a custom struct. Could be a problem when we're sending all unread messages, since this would probably be a list of lists or a list of dicts or a list of "Message"s; or rather, it would introduce more overhead because we first need to send back the IDs of the messages and then request each of them individually. Might be something we're willing to compromise on
        - We could do a similar thing as with the lists and define an "object" flag which is then interpreted as a dictionary with custom types; for this, we also need to allow lists of objects or more generally, lists of different types
        - This is fine in Python but could cause problems in other languages like C


- How do you specify which messages you want to delete?
    - Could go by message ID individually
    - Could go by sender/receiver name

### General Notes
- Each request should carry a token to indicate that the user is logged in and to map the request to the correct account. Do we refresh this token? How often? Does the server just send back the new token with each request? In that case, what do we do if a packet is lost; this would automatically log the user out

### User Interface Design

- How do we display the unread messages?
    - VBS: Maybe users want to be able to open the app without reading *any* messages, so we should only load messages on request
    - We can either group them by conversation or just give a stream of messages ordered by timestamp
    - Think: "Refresh mail" button in any standard Mail program
- To send a message, should users be able to click on a conversation and send a message there or do we just have two input fields, one for the user name and one for the message, and then you just have to input the user name every time

### Data Storage
- How do we want to store the data? Should we just keep stuff in memory, should we use an actual database (e.g. sqlite) or just keep stuff in one or multiple JSON files?

VBS says:

1. first check if account exists, if yes then check hash of password matches, (send string of hash and username over network); if no then client shows an error of account does not exist then allow to put in password (which client then hashes). if successful then always use username password hash as our token (extension is valid for some period of time)

2. basically the same as above; check if account exists and check if password match. also send back the number of unread messages

3. send search pattern, what page we are on, how big are the pages; 

4. specify the person through user ID or username, we need to keep track of the sockets belongs to which user, once destroyed also keep track of who is destroyed; we will store everything when it is not logged in, otherwise send immediately; get some form of confirmation all good back

5. send back bunch of mesages at once using the object thing; what if the message is super long? (think about this). be able to send a request to the server of how many messages we want, if there are none just send back something; 

6. delete a specific message - needs an ID then client interprets and checks if you can delete this message; also the similar for a list of messages; for a whole conversation - separate action for deleting conversation with a username/userID; delete account - can only delete your account, what happens to our messages? (policy: every message you send and sent to you are gone)

7. front end



To Do:

1. make a small library that takes data and turn into wireprotocl (bytes); and translate it back; probably use UTS-8 for strings interpretation

2. user interface, graphic interface for the client
- how do we display messages, unread messages gets added to the top, all previous read messages at the bottom(let us not do conversations)

- when you log in, you see all previous messages, and shows how many unread messages, and you can ask for some number of unread messages; do this in pages, 

3. server side + database storage


## Code Ideas

Some ideas for what the field ID to field mapping could look like so that the data becomes a bit easier to use for the programmer and we don't need to know all of the field IDs by heart:

In [ ]:
key_to_field_id = {
    "username": 0,
    "password_digest": 1,
}

key_to_field_type = {
    "username": str,
    "password_digest": str,
    "asdkjh": int,
}

def get(data, field: str, default = None):
    if field in key_to_field_id:
        return data[key_to_field_id[id]]
    
    return default

In [ ]:
protocol:
    field ID, field type, field size, field content
    1 byte, 

In [ ]:
0x20 -> list
0x21 -> list of int
0x22 -> list of str

In [ ]:
class Message:
    id: int
    content: str
    from_id: int
    to_id: int
    timestamp: datetime

In [ ]:
{
    "version": 1,
    "operation": "list",
    "arguments": {
        "search": "hello"
    }
}

In [ ]:
what the request would look like with JSON:

request

{
    "version": 1,
    "operation": "messages",
    "arguments": {
        "number": 3
    }
}

response 

{
    "version": 1,
    "status": "success",
    "items": [
        {
            "message_id": 1,
            "sender": "user_1",
            "message": "hey",
            "timestamp": "2025-01-01"
        },
        {
            "message_id": 2,
            "sender": "user_1",
            "message": "what's up?",
            "timestamp": "2025-01-01"
        },
        {
            "message_id": 3,
            "sender": "user_2",
            "message": "yooo",
            "timestamp": "2025-01-01"
        }
    ],
    "remaining": 2
}


In [ ]:
what the same request would look like with our protocol:

request: ..., operation = messages, quantity = 3
response: ..., items = [1, 2, 3], remaining = 2
request: ..., operation = message_detail, id = 1
response: ..., message = ["user_1", "hey", "2025-01-01"]
request: ..., operation = message_detail, id = 2
response: ..., message = ["user_1", "what's up?", "2025-01-01"]
request: ..., operation = message_detail, id = 3
response: ..., message = ["user_2", "yooo", "2025-01-01"]

In [ ]:
{
    "messages": [
        {
            "id": 1,
            "content": "My message",
            "from_id": 2,
            "to_id": 3,
            "timestamp": "2025-01-01"
        }
    ],
    "users": [
        {
            "id": 
        }
    ]
}

In [ ]:
message = [
    "user_name",
    "message",
    "timestamp"
]

In [ ]:
request: give me the messages
response: here are the message IDs (all integers)
for each message ID:
    request: give me the message (message ID)
    response: here is the message (list of str)

In [ ]:
send("message", {"to": "test_user", "message": "hello"})

In [ ]:
"hello"

message -> 3
field_type -> str
field_length -> 5

Okay, let's not do that. Instead, if we support dictionaries as part of our protocol, we can get rid of the field ID encoding and just use field names.

As a separate note, could there be concurrency issues with one client waiting for a response while the server sends a push message to another client? I don't think so, since our server is not multithreaded.

VBS: Okay so it turns out that the requirement that the server can just push a message to the client whenever it wants is actually a bit more challenging than i thought, or at least it makes the rest of the communication a bit more challenging. Basically the problem is that when i have a separate thread listening to data from the server at all times, it can happen that it takes away the response from another request that is waiting for a response. then that request is blocked because it’s waiting for a response, and the thread checking for incoming data doesn’t know what to do with the data it just got. So this means we need to do requests completely asynchronously: send it and then wait until something is in a queue and tell the socket managing the connection that if it gets something of the correct format, it should give it to this request.

I can think of two solutions to this: either make responses identifiable using similar/the same request codes so that we can have the client listening to everything from the server and then decide what to do with the response. or we can make it so that whenever the client has just sent a request, they simply tell the listening thread to stop listening until it has received a response.

Update: I just tried implementing the latter because it should be pretty quick, but you get weird race conditions between the listening thread stopping and the other thread sending + receiving.

### Updated Wire Protocol
To facilitate both standard request/response communication and server-push communication over the same socket, we use a separate listener thred on the client which takes care of receiving and routing the data from the server. This also means that we need to be able to identify the responses from the server (i.e. what request they correspond to). For this, we modify our header and replace the `flags` variable with `request_id`, treated as an unsigned integer of 2 bytes. This gives the updated header:

Header (8 bytes):
- Version (1 byte) -- for future compatibility
- Operation/status code (1 byte) -- for correct routing
- Request ID (2 bytes) -- to identify responses to each request
- Packet checksum (2 bytes) -- to verify packet payload
- Payload length (2 bytes) -- to know when to stop loading

## Comparison JSON vs. Custom Wire Protocol

Before we do our comparison, here is a quick prediction of what we think is going to happen:
- Both our protocol and JSON are very flexible (in fact, our protocol can encode everything JSON can and more), so in terms of the amount of data sent, we expect them to be roughly similar
    - In fact, while our ability to encode byte streams should be more efficient than first converting them to string such that JSON can parse them, in order to allow for seamless switching between our custom wire protocol and JSON, we actually need to treat the password hash and tokens as strings such that we lose some of the storage efficiency of our custom wire protocol
    - Nonetheless, we believe that our wire protocol will use slightly less space than JSON
- In terms of speed, while our custom wire protocol uses low-level byte operations, it is likely not as optimized as the heavily-used JSON parser that ships with Python. Thus, we expect our custom wire to take a little longer in the encoding and decoding, and since the messages are sent on the local network and we don't expect the differences in encoding size to be too large between JSON and our custom protocol, we expect our own wire protocol to actually be a bit slower than JSON

To do the test, we average the time between creating a request and parsing the response over multiple iterations back-to-back. We do this both for login calls and sending messages with randomly-generated 128-character strings.

In [ ]:
import secrets
import socket

from src.common import Request, OP_TO_CODE, hash_password

TOKEN = ""
PASSWORD_1 = hash_password(secrets.token_hex(16))
PASSWORD_2 = hash_password(secrets.token_hex(16))

def test_env_setup(sock: socket.socket):
    # Create the necessary accounts
    request_1 = Request(OP_TO_CODE["register"], {
        "username": "user_1",
        "password_hash": PASSWORD_1,
    })
    request_1.push(sock)

    response_1 = Request.receive(sock)

    request_2 = Request(OP_TO_CODE["register"], {
        "username": "user_2",
        "password_hash": PASSWORD_2,
    })
    request_2.push(sock)

    response_2 = Request.receive(sock)

def test_fn_login(sock: socket.socket):
    request = Request(OP_TO_CODE["login"], {
        "username": "user_1",
        "password_hash": PASSWORD_1,
    })
    request.push(sock)

    response = Request.receive(sock)
    return response.data["token"]

def test_fn_message(sock: socket.socket):
    request = Request(OP_TO_CODE["message"], {
        "to": "user_2",
        "content": secrets.token_hex(64),
        "token": TOKEN,
    })
    request.push(sock)

    return Request.receive(sock)

In [ ]:
# Assume server is running on localhost with port 23456
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect(("localhost", 23456))

In [ ]:
# Create users
test_env_setup(sock)

In [ ]:
%%timeit
test_fn_login(sock)

In [ ]:
TOKEN = test_fn_login(sock)

In [ ]:
%%timeit
test_fn_message(sock)

In [ ]:
sock.close()

### Results: Custom Wire Protocol

- Create account: sent 114 bytes, received 86 bytes.
- Login: sent 114 bytes, received 99 bytes. 7 runs, 1,000 loops each: 1.34ms ± 52.2 µs
- Send message: sent 241 bytes, received 238 bytes. 7 runs, 100 loops each: 2.23 ms ± 96.4 μs

### Results: JSON

- Create account: sent 115 bytes, received 85 bytes.
- Login: sent 115 bytes, received 98 bytes. 7 runs, 1,000 loops each: 1.25 ms ± 15.8 μs
- Send message: sent 244 bytes, received 243 bytes. 7 runs, 100 loops each: 2.17 ms ± 117 μs

We see that our predictions were correct! Indeed, the JSON encoding tends to be ever so slightly larger while still being a little bit faster, likely due to optimized encoding and decoding. It would be interesting to check how much of a difference it makes that we chose a little-endian based protocol.

For a larger system, there are a few considerations here. For one, if it suffices to only support our custom wire protocol and we send a fair amount of pure bytestreams (e.g. as tokens), then our custom wire protocol might end up being a good bit smaller and hence more efficient. This becomes especially relevant as the networks become bigger and individual components are spatially further apart or as we send bigger files: if, for instance, we wanted to support sending images/videos, then it would be much easier (and more efficient) to send this as a bytestream rather than through JSON, which was obviously not made to support images. We chose to use a more flexible encoding system as opposed to hardcoding fields, which would have allowed us to use an even more efficient length-value encoding with the typing implemented by the server and client based on a shared understanding of what each field means.

In terms of parsing efficiency, it appears that JSON is a little bit faster than our encoder/decoder. To improve performance here, one could use a compiled language like C/C++ to do the encoding/decoding, and then use Python for the rest. 